# 📊 Retriever Performance Comparison (N=5 Trials)

이 노트북은 **세 가지 리트리버(BM25, Vector, Hybrid)**의 성능을 **5회의 독립적인 시행**을 통해 비교 분석합니다.

### 🧪 실험 조건
- **Dataset**: `golden_dataset.json` (총 3,000+개 중 무작위 추출)
- **Trials**: 5회 반복 (각 시행마다 **동일한 50개 샘플**로 3개 모델 모두 테스트)
- **Metrics**: 
  - **Hit@5**: 상위 5개 결과 내 정답 포함 여부
  - **MRR**: Mean Reciprocal Rank (정답 순위의 효율성)
- **Visualization**: Trial별 성능 변화 및 전체 평균 비교 차트

In [ ]:
# ✅ Parameters
N_TRIALS = 5
SAMPLE_SIZE_PER_TRIAL = 50
K_VALUE = 5

# Setup
import sys
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import json
import random
import matplotlib.pyplot as plt
import seaborn as sns
import asyncio

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

load_dotenv(os.path.join(project_root, ".env"))
print(f"Project Root: {project_root}")

# 한글 폰트 설정 (Mac OS) - 그래프 깨짐 방지
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# Initialize Retrievers
from src.retrieval.bm25_retriever import BM25Retriever
from src.retrieval.vector_retriever import VectorRetriever
from src.retrieval.hybrid_search import HybridRetriever

print("🚀 Initializing Retrievers...")
bm25 = BM25Retriever(version="v3", collection_name="care_guides")
vector = VectorRetriever(version="v3", collection_name="care_guides")
hybrid = HybridRetriever(version="v3", collection_name="care_guides")
print("✅ All Retrievers Ready.")

In [ ]:
# Load Dataset
dataset_path = os.path.join(project_root, "data/v3/golden_dataset.json")

if os.path.exists(dataset_path):
    with open(dataset_path, "r", encoding="utf-8") as f:
        FULL_DATASET = json.load(f)
    print(f"📚 Loaded {len(FULL_DATASET)} total cases.")
else:
    raise FileNotFoundError("Dataset not found!")

In [ ]:
# 🧪 Evaluation Function
async def evaluate(retriever, dataset, name, k=5):
    hits = 0
    reciprocal_ranks = 0
    
    for item in dataset:
        query = item.get("query")
        target = item.get("expected_keyword", item.get("target"))
        
        try:
            specialist = item.get("specialist")
            results = await retriever.search(query, specialist=specialist, limit=k)
        except Exception:
            results = []
            
        found_rank = None
        for rank, res in enumerate(results):
            content = (
                res.get('name_ko', '') + " " + 
                res.get('title_refined', '') + " " + 
                res.get('text', '')
            ).lower()
            
            if target.lower() in content:
                found_rank = rank + 1
                break
        
        if found_rank:
            hits += 1
            reciprocal_ranks += 1.0 / found_rank
            
    return {
        "Model": name,
        "Hit@5": hits / len(dataset),
        "MRR": reciprocal_ranks / len(dataset)
    }

In [ ]:
# 🔄 Run Experiment Loop
results_log = []

print(f"🏁 Starting {N_TRIALS} Trials...")

for i in range(1, N_TRIALS + 1):
    print(f"\n[Trial {i}/{N_TRIALS}] Sampling Data...")
    
    # 1. Random Sampling (Same dataset for all models in this trial)
    trial_data = random.sample(FULL_DATASET, min(SAMPLE_SIZE_PER_TRIAL, len(FULL_DATASET)))
    
    # 2. Evaluate Models
    res_bm25 = await evaluate(bm25, trial_data, "BM25", k=K_VALUE)
    res_vector = await evaluate(vector, trial_data, "Vector", k=K_VALUE)
    res_hybrid = await evaluate(hybrid, trial_data, "Hybrid", k=K_VALUE)
    
    # 3. Log Results
    for res in [res_bm25, res_vector, res_hybrid]:
        res["Trial"] = i
        results_log.append(res)
        print(f"  > {res['Model']}: Hit@5={res['Hit@5']:.2f}, MRR={res['MRR']:.2f}")
        
df_results = pd.DataFrame(results_log)

## 📊 Visualization

In [ ]:
# Aggregated Stats
summary = df_results.groupby("Model")[["Hit@5", "MRR"]].agg(["mean", "std"])
display(summary)

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 1. Hit@5 Comparison
sns.barplot(data=df_results, x="Model", y="Hit@5", ax=axes[0], palette="viridis", errorbar="sd")
axes[0].set_title("Hit@5 Score Comparison (Mean ± SD)")
axes[0].set_ylim(0, 1.0)
for p in axes[0].patches:
    axes[0].annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha = 'center', va = 'bottom', xytext = (0, 5), textcoords = 'offset points')

# 2. MRR Comparison
sns.barplot(data=df_results, x="Model", y="MRR", ax=axes[1], palette="magma", errorbar="sd")
axes[1].set_title("MRR Score Comparison (Mean ± SD)")
axes[1].set_ylim(0, 1.0)
for p in axes[1].patches:
    axes[1].annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha = 'center', va = 'bottom', xytext = (0, 5), textcoords = 'offset points')

plt.tight_layout()
plt.show()

In [ ]:
# Trial-wise Line Plot (Consistency Check)
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(data=df_results, x="Trial", y="MRR", hue="Model", marker="o", ax=ax)
ax.set_title("MRR Stability Across 5 Trials")
ax.set_ylim(0, 1.0)
ax.set_xticks(range(1, N_TRIALS + 1))
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()